In [1]:
import API_Call
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np

df = pd.read_csv("data/resale-flat-prices/cleaned_prices.csv")
df.head()

,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,resale_price,year_sold,mth_sold,age_sold
0,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,IMPROVED,9000.0,1990,1,13.0
1,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,04 TO 06,31.0,IMPROVED,6000.0,1990,1,13.0
2,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,IMPROVED,8000.0,1990,1,13.0
3,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,07 TO 09,31.0,IMPROVED,6000.0,1990,1,13.0
4,ANG MO KIO,4 ROOM,211,ANG MO KIO AVE 3,04 TO 06,81.0,NEW GENERATION,80000.0,1990,1,13.0


In [2]:
df['block'] = df['block'].str.strip()
df['street_name'] = df['street_name'].str.strip()

df['address'] = df['block'] + ' ' + df['street_name']
addresses = pd.DataFrame(df['address'])
addresses.drop_duplicates(inplace=True)
addresses.reset_index(drop=True, inplace=True)
addresses.head()

,address
0,309 ANG MO KIO AVE 1
1,211 ANG MO KIO AVE 3
2,209 ANG MO KIO AVE 3
3,231 ANG MO KIO AVE 3
4,308 ANG MO KIO AVE 1


In [3]:
len(addresses)

9502

In [4]:
#default latitude/longitude
addresses['lat'] = np.nan
addresses['long'] = np.nan

API_Call.process_df(addresses, "address")

In [5]:
#check any invalid address/request
addresses.isna().sum()

address      0
lat        117
long       117
dtype: int64

In [6]:
addresses[addresses['lat'].isna()]
#manual random inspection reveals some of these addresses to be outdated and some could not be found in one map api

,address,lat,long
0,309 ANG MO KIO AVE 1,NaN,NaN
127,4 HILLVIEW AVE,NaN,NaN
145,33 TAMAN HO SWEE,NaN,NaN
151,111 BT MERAH VIEW,NaN,NaN
173,22 OUTRAM PK,NaN,NaN
...,...,...,...
9234,51 TANGLIN HALT RD,NaN,NaN
9254,89 ZION RD,NaN,NaN
9280,137 ALEXANDRA RD,NaN,NaN
9297,11 REDHILL CL,NaN,NaN


In [7]:
#merge df
merged = pd.merge(df, addresses, how='left', on='address')
merged.drop('address', axis=1, inplace=True)
merged.head(10)

,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,resale_price,year_sold,mth_sold,age_sold,lat,long
0,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,IMPROVED,9000.0,1990,1,13.0,NaN,NaN
1,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,04 TO 06,31.0,IMPROVED,6000.0,1990,1,13.0,NaN,NaN
2,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,IMPROVED,8000.0,1990,1,13.0,NaN,NaN
3,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,07 TO 09,31.0,IMPROVED,6000.0,1990,1,13.0,NaN,NaN
4,ANG MO KIO,4 ROOM,211,ANG MO KIO AVE 3,04 TO 06,81.0,NEW GENERATION,80000.0,1990,1,13.0,1.369196965617,103.841666636086
5,ANG MO KIO,4 ROOM,209,ANG MO KIO AVE 3,07 TO 09,91.0,NEW GENERATION,72000.0,1990,1,13.0,1.36897079641509,103.843284524871
6,ANG MO KIO,4 ROOM,231,ANG MO KIO AVE 3,04 TO 06,92.0,NEW GENERATION,72000.0,1990,1,13.0,1.36856103572357,103.837525234344
7,ANG MO KIO,4 ROOM,308,ANG MO KIO AVE 1,10 TO 12,92.0,NEW GENERATION,86000.0,1990,1,14.0,1.36569665029696,103.844347269095
8,ANG MO KIO,4 ROOM,254,ANG MO KIO AVE 4,10 TO 12,97.0,NEW GENERATION,72000.0,1990,1,13.0,1.37020344042146,103.834687722549
9,ANG MO KIO,4 ROOM,103,ANG MO KIO AVE 3,07 TO 09,90.0,NEW GENERATION,64500.0,1990,1,12.0,1.37101657804604,103.839304839187


In [8]:
len(merged)

875236

In [9]:
#address not found, block possibly already en-bloc
merged.dropna(inplace=True)
len(merged)

865077

In [10]:
#save file
merged.to_csv("data/coords/coords_HDB.csv", index=False)